In [1]:
# Instal dependency
# !pip install open_clip_torch

In [2]:
# Import module
import os
import open_clip
import glob
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm

# Parse data path

In [3]:
keyframes_dir = '../keyframe_information/keyframe'
all_keyframe_paths = dict()
for part in sorted(os.listdir(keyframes_dir)):
    data_part = part.split('_')[-1] # L01, L02 for ex
    all_keyframe_paths[data_part] =  dict()

for data_part in sorted(all_keyframe_paths.keys()):
    data_part_path = f'{keyframes_dir}/{data_part}'
    video_dirs = sorted(os.listdir(data_part_path))
    video_ids = [video_dir.split('_')[-1] for video_dir in video_dirs]
    for video_id, video_dir in zip(video_ids, video_dirs):
        keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.jpg'))
        all_keyframe_paths[data_part][video_id] = keyframe_paths

print(all_keyframe_paths)

{'Video0': {'L00': ['../keyframe_information/keyframe/Video0/L00/keyframe_10_44.611233.jpg', '../keyframe_information/keyframe/Video0/L00/keyframe_11_52.952900.jpg', '../keyframe_information/keyframe/Video0/L00/keyframe_12_58.825433.jpg', '../keyframe_information/keyframe/Video0/L00/keyframe_13_67.167100.jpg', '../keyframe_information/keyframe/Video0/L00/keyframe_14_73.573500.jpg', '../keyframe_information/keyframe/Video0/L00/keyframe_15_77.811067.jpg', '../keyframe_information/keyframe/Video0/L00/keyframe_16_86.152733.jpg', '../keyframe_information/keyframe/Video0/L00/keyframe_17_88.988900.jpg', '../keyframe_information/keyframe/Video0/L00/keyframe_18_90.457033.jpg', '../keyframe_information/keyframe/Video0/L00/keyframe_1_0.000000.jpg', '../keyframe_information/keyframe/Video0/L00/keyframe_2_2.502500.jpg', '../keyframe_information/keyframe/Video0/L00/keyframe_3_3.603600.jpg', '../keyframe_information/keyframe/Video0/L00/keyframe_4_4.804800.jpg', '../keyframe_information/keyframe/Video

# Model

In [4]:
##### Load Model #####
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', device=device, pretrained='datacomp_xl_s13b_b90k')

cpu


/opt/anaconda3/lib/python3.12/site-packages/open_clip/factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=map

In [5]:
bs = 4
save_dir = './CLIPv2_features'
if not os.path.exists(save_dir):
  os.mkdir(save_dir)

for key, video_keyframe_paths in tqdm(all_keyframe_paths.items()):
    video_ids = sorted(video_keyframe_paths.keys())
    
    if not os.path.exists(os.path.join(save_dir, key)):
        os.mkdir(os.path.join(save_dir, key))
    
    for video_id in tqdm(video_ids):
        video_feats = []
        video_keyframe_path = video_keyframe_paths[video_id]
        for i in range(0, len(video_keyframe_path), bs):
            # Support batchsize inferencing
            images = []
            image_paths = video_keyframe_path[i:i+bs]
            for image_path in image_paths:
                image = preprocess(Image.open(image_path)).unsqueeze(0)
                images.append(image)
            images = torch.cat(images).to(device)

            with torch.no_grad(), torch.cuda.amp.autocast():
                image_feats = model.encode_image(images)
            image_feats /= image_feats.norm(dim=-1, keepdim=True)

            for b in range(image_feats.shape[0]):
                video_feats.append(image_feats[b].detach().cpu().numpy().astype(np.float32).flatten())
        
        np.save(f'{save_dir}/{key}/{video_id}.npy', video_feats)

  0%|          | 0/5 [00:00<?, ?it/s]/var/folders/fw/f3018xmn4d91q9730vjl_p_r0000gn/T/ipykernel_74903/1485413450.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
/opt/anaconda3/lib/python3.12/site-packages/torch/amp/autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
100%|██████████| 5/5 [01:19<00:00, 15.85s/it]
